In [15]:
# arxiv--research
# tools-creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [25]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [17]:
# used the inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
# to run this wrraper
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
print(wiki.name)

wikipedia


In [18]:
# used the inbuilt tool of arxiv
arxiv_wrapper_api = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max = 250)
arxiv = ArxivQueryRun(api_wrapper = arxiv_wrapper_api)
print(arxiv.name)

arxiv


In [19]:
# if I want to combine both of these tools
tools = [wiki,arxiv]

In [20]:
# I want to create tools [rag tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

In [21]:
# load the webpage
loader = WebBaseLoader("https://www.langchain.com/langsmith")
docs = loader.load()
# split the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=250)
split_docs = text_splitter.split_documents(docs)
# create the vector store
vectorstore = FAISS.from_documents(split_docs,embedding = HuggingFaceEmbeddings())
# Now use the retriever to retrieve the relavant documents
retriever = vectorstore.as_retriever()

In [22]:
# To create the tool, convert the retriever into a tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever = retriever, name = "Langsmith-search", description = "useful for when you want to find information about langsmith")
print(retriever_tool.name)

Langsmith-search


In [23]:
tools = [wiki,arxiv,retriever_tool]

In [26]:
# Run all the tools and agents with LLM model

from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")

In [27]:
# prompt template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [39]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm = llm, tools= tools, prompt=prompt)
print(agent)

first=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
}) middle=[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hu

In [40]:
# Agent Execution
from langchain.agents import AgentExecutor
agentexecutor = AgentExecutor(agent = agent,tools = tools, verbose = True)
print(agentexecutor)


verbose=True agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hu

In [41]:
agentexecutor.invoke({"input" : "Tell me about langsmith"})


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `Langsmith-search` with `{'query': 'langsmith'}`


Yes, you can log traces to LangSmith using a standard OpenTelemetry client to access all LangSmith features, including tracing, running evals, and prompt engineering. See the docs.How can LangSmith help with observability and evaluation?

LangSmith traces contain the full information of all the inputs and outputs of each step of the application, giving users full visibility into their agent or LLM app behavior. LangSmith also allows users to instantly run evals to assess agent or LLM app performance — including LLM-as-Judge evaluators for auto-scoring and the ability to attach human feedback. Learn more.I can’t have data leave my environment. Can I self-host LangSmith?

Yes, we allow customers to self-host LangSmith on our enterprise plan. We deliver the software to run on your Kubernetes cluster, and data will not leave your environment. For more information, check out our documentation.Where is LangSmith data stored?

feed

{'input': 'Tell me about langsmith',
 'output': "It seems that LangSmith is a platform designed for observability and evaluation of AI applications, particularly those built using LangChain. It provides features such as tracing, running evals, and prompt engineering, allowing developers to debug, test, and monitor their AI app performance. LangSmith also allows users to self-host the platform on their own Kubernetes cluster, ensuring that data remains within their environment.\n\nIt's also clear that LangSmith is designed to be flexible and works with various programming languages, including Python and TypeScript. Additionally, LangSmith does not add any latency to the application and does not train on user data, giving users full control over their data.\n\nOverall, LangSmith seems to be a powerful tool for developers building AI applications, providing them with the necessary insights to improve their app's performance and reliability."}

In [35]:
print(retriever_tool.run("Tell me about langsmith"))

Yes, you can log traces to LangSmith using a standard OpenTelemetry client to access all LangSmith features, including tracing, running evals, and prompt engineering. See the docs.How can LangSmith help with observability and evaluation?

LangSmith traces contain the full information of all the inputs and outputs of each step of the application, giving users full visibility into their agent or LLM app behavior. LangSmith also allows users to instantly run evals to assess agent or LLM app performance — including LLM-as-Judge evaluators for auto-scoring and the ability to attach human feedback. Learn more.I can’t have data leave my environment. Can I self-host LangSmith?

Yes, we allow customers to self-host LangSmith on our enterprise plan. We deliver the software to run on your Kubernetes cluster, and data will not leave your environment. For more information, check out our documentation.Where is LangSmith data stored?

For Cloud SaaS, traces are stored in GCP us-central-1 or GCP europ